In [4]:
from __future__ import absolute_import

import pandas as pd
import numpy as np
import dask.dataframe as dd

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, r2_score


df = dd.read_csv('D:/Dados historicos-NOVO/WINFUT_01042019_31122019/WIN$N_BMF_T_bid_ask.csv', sep='\t', usecols=[0, 1, 4, 5])
# df = dd.read_csv('D:/Dados historicos-NOVO/Bovespa_02012017_30062021/SEQL3_BOV_T_bid_ask.csv', sep='\t', usecols=[4, 5])

df = df.dropna().reset_index(drop=True)

In [5]:
def RSI(data, period, applied_price):
    dataframe = data

    delta = dataframe[applied_price].diff(1)
    delta = delta.fillna(0)

    dataframe['<DELTA>'] = delta

    dataframe['<UP>'] = delta.copy()
    dataframe['<DOWN>'] = delta.copy()

    
#     dataframe['<UP>'] = dataframe['<UP>'].apply(lambda x: x if x < 0 else 0)
    for _i in range(len(dataframe)):
        if dataframe['<DELTA>'].iloc[_i] < 0:
            dataframe['<UP>'].iloc[_i] = 0
            dataframe['<DOWN>'].iloc[_i] = dataframe['<DELTA>'].iloc[_i]
        elif dataframe['<DELTA>'].iloc[_i] > 0:
            dataframe['<UP>'].iloc[_i] = dataframe['<DELTA>'].iloc[_i]
            dataframe['<DOWN>'].iloc[_i] = 0
        else:
            dataframe['<UP>'].iloc[_i] = 0
            dataframe['<DOWN>'].iloc[_i] = 0
    
#     dataframe['<DOWN>'] = dataframe['<DOWN>'].apply(lambda x: x if x > 0 else 0)

    # up = dd.DataFrame(delta.copy())
    # down = dd.DataFrame(delta.copy())


    # up[up < 0] = 0
    # down[down > 0] = 0
    
    # dataframe = dataframe.assign(temp_f=up).fillna(0)
    # dataframe = dataframe.assign(temp_k=down).fillna(0)

    # dataframe = dataframe.rename(columns={'temp_f':'<UP>', 'temp_k':'<DOWN>'})

    avg_gain = dataframe['<UP>'].rolling(window=period).mean()
    avg_loss = abs(dataframe['<DOWN>'].rolling(window=period).mean())

    RS = avg_gain/avg_loss

    RSI = 100.0 - (100.0/(1.0 + RS))

    dataframe['<INDICATOR>'] = RSI

    return dataframe

In [6]:
df = RSI(df, 7, '<LAST>')

AttributeError: 'Series' object has no attribute 'iloc'

In [ ]:
df['<INDICATOR>'][500:530].compute()